# Lecture 5

 Fall 2023

A demonstration of advanced `pandas` syntax to accompany Lecture 5.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import plotly.express as px

## More on `Groupby`

### Slido Exercise

Try to predict the results of the `groupby` operation shown. The answer is below the image.

<img src="/content/drive/MyDrive/groupby.png" alt="Image" width="600">

The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd".

In [3]:
# Form a data frame using dictionary
# Answer Here
ds = pd.DataFrame(dict(x=[3, 1, 4, 1, 5, 9, 2, 5, 6],
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']),
                      index=list('ABCABCACB') )
ds

,x,y
A,3,ak
B,1,tx
C,4,fl
A,1,hi
B,5,mi
C,9,ak
A,2,ca
C,5,sd
B,6,nc


In [14]:
#Use groupby on index and get max of each group
grouped_max = ds.groupby(ds.index).max()
print(grouped_max)

   x   y
A  3  hi
B  6  tx
C  9  sd


### Loading `babynames` Dataset

In [2]:
import pandas as pd
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.tail(10)

,State,Sex,Year,Name,Count
407418,CA,M,2022,Zach,5
407419,CA,M,2022,Zadkiel,5
407420,CA,M,2022,Zae,5
407421,CA,M,2022,Zai,5
407422,CA,M,2022,Zay,5
407423,CA,M,2022,Zayvier,5
407424,CA,M,2022,Zia,5
407425,CA,M,2022,Zora,5
407426,CA,M,2022,Zuriel,5
407427,CA,M,2022,Zylo,5


### Case Study: Name "Popularity"

In this exercise, let's find the name with sex "F" that has dropped most in popularity since its peak usage in California. We'll start by filtering `babynames` to only include names corresponding to sex "F".

In [3]:
# Select the names only. of baby grils
import pandas as pd

f_babynames = babynames[babynames["Sex"] == "F"]

peak_counts = f_babynames.groupby("Name")["Count"].max()


percentage_drop = (peak_counts - f_babynames.groupby("Name")["Count"].last()) / peak_counts * 100

print(percentage_drop)
most_dropped_name = percentage_drop.idxmax()

print("The female baby name that has dropped the most in popularity since its peak usage is:", most_dropped_name)
#print(percentage_drop)

Name
Aadhini     0.000000
Aadhira    50.000000
Aadhya     34.000000
Aadya      41.379310
Aahana     73.076923
             ...    
Zyanya     53.333333
Zyla        0.000000
Zylah       0.000000
Zyra        0.000000
Zyrah      16.666667
Name: Count, Length: 13782, dtype: float64
The female baby name that has dropped the most in popularity since its peak usage is: Debra


In [4]:
# We sort the data by year
babynames.sort_values(by="Year")

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
239654,CA,M,1910,Franklin,5
239655,CA,M,1910,Guadalupe,5
239656,CA,M,1910,Leland,5
239657,CA,M,1910,Mario,5
...,...,...,...,...,...
237072,CA,F,2022,Johana,22
237073,CA,F,2022,Kit,22
237074,CA,F,2022,Korra,22
237076,CA,F,2022,Lavender,22


To build our intuition on how to answer our research question, let's visualize the prevalence of the name "Jennifer" over time.

In [5]:
# We'll talk about how to generate plots in a later lecture
fname=babynames[babynames["Sex"]=="F"]
fname=fname.sort_values(["Year"])
jenn=fname[fname["Name"]=="Jennifer"]["Count"]
maxjen=max(jenn)
currjenn=jenn.iloc[-1]
rtp=maxjen/currjenn
print(rtp)

53.20175438596491


We'll need a mathematical definition for the change in popularity of a name in California.

Define the metric "Ratio to Peak" (RTP). We'll calculate this as the count of the name in 2022 (the most recent year for which we have data) divided by the largest count of this name in *any* year.

A demo calculation for Jennifer:

In [6]:
# In the year with the highest Jennifer count, 6065 Jennifers were born
import pandas as pd

# Assuming 'babynames' is your DataFrame with 'Name', 'Year', and 'Count' columns
# Load your DataFrame here if it's not already loaded

# Define the peak count for Jennifer
peak_count_jennifer = 6065

# Find the count for Jennifer in 2022
count_jennifer_2022 = babynames[(babynames['Name'] == 'Jennifer') & (babynames['Year'] == 2022)]['Count'].iloc[0]

# Calculate the Ratio to Peak (RTP)
rtp_jennifer = count_jennifer_2022 / peak_count_jennifer

print(f"The Ratio to Peak (RTP) for Jennifer is: {rtp_jennifer}")


The Ratio to Peak (RTP) for Jennifer is: 0.018796372629843364


In [7]:
# Remember that we sorted f_babynames by year.
# This means that grabbing the final entry gives us the most recent count of Jennifers: 114
# In 2022, the most recent year for which we have data, 114 Jennifers were born
# Peak count for Jennifer
peak_count_jennifer = 6065

# Most recent count for Jennifer in 2022
recent_count_jennifer = 114

# Calculate the Ratio to Peak (RTP)
rtp_jennifer = recent_count_jennifer / peak_count_jennifer

print(f"The Ratio to Peak (RTP) for Jennifer is: {rtp_jennifer:.4f}")



The Ratio to Peak (RTP) for Jennifer is: 0.0188


In [26]:
# Compute the RTP
fname=babynames[babynames["Sex"]=="F"]
fname=fname.sort_values(["Year"])
jenn=fname[fname["Name"]=="Jennifer"]["Count"]
maxjen=max(jenn)
currjenn=jenn.iloc[-1]
rtp=maxjen/currjenn
print(rtp)

53.20175438596491


We can also write a function that produces the `ratio_to_peak`for a given `Series`. This will allow us to use `.groupby` to speed up our computation for all names in the dataset.

In [31]:
def ratio_to_peak(series):
    
    #Compute the RTP for a Series containing the counts per year for a single name
    return series.iloc[-1]/ max(series)

jenn=fname[fname["Name"]=="Jennifer"]["Count"]
ratio_to_peak(jenn)

0.018796372629843364

In [32]:
# Construct a Series containing our Jennifer count data

# Then, find the RTP
jenn=fname[fname["Name"]=="Jennifer"]["Count"]
ratio_to_peak(jenn)

0.018796372629843364

Now, let's use `.groupby` to compute the RTPs for *all* names in the dataset.

You may see a warning message when running the cell below. As discussed in the lecture, `pandas` can't apply an aggregation function to non-numeric data (it doens't make sense to divide "CA" by a number). We can select numerical columns of interest directly.

In [39]:
rtp_table = f_babynames.groupby("Name")[["Year", "Count"]].agg(ratio_to_peak)
rtp_table

,Year,Count
Name,,
Aadhini,1.0,1.000000
Aadhira,1.0,0.500000
Aadhya,1.0,0.660000
Aadya,1.0,0.586207
Aahana,1.0,0.269231
...,...,...
Zyanya,1.0,0.466667
Zyla,1.0,1.000000
Zylah,1.0,1.000000


In [40]:
# Results in a TypeError
rtp_table = f_babynames.groupby("Name").agg(ratio_to_peak)
rtp_table

TypeError: unsupported operand type(s) for /: 'str' and 'str'

### Slido Exercise

Is there a row where `Year` is not equal to 1?

In [41]:
# Find Unique values in the Year column of rtp_table dataframe
unique=rtp_table.value_counts("Year")
unique

Year
1.0    13782
Name: count, dtype: int64

In [63]:
# Dropping the Year column
#babynames.drop("Year", axis=1, inplace=True)

In [57]:
# Rename "Count" to "Count RTP" for clarity
aq=babynames.rename(columns={"Count":"Count RTP"})

In [ ]:
# What name has fallen the most in popularity?
#Debra

In [55]:
import plotly.express as px  # Correctly import Plotly Express

def plot_name(*names):
    # Ensure f_babynames is defined and has the columns 'Name', 'Year', and 'Count'
    fig = px.line(f_babynames[f_babynames["Name"].isin(names)],
                  x="Year", y="Count", color="Name",
                  title=f"Popularity for: {', '.join(names)}")
    fig.update_layout(font_size=18, width=1000, height=400)
    return fig

# Call the function with the name 'Debra'
plot_name("Debra")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Name=Debra<br>Year=%{x}<br>Count=%{y}<extra></extra>',
              'legendgroup': 'Debra',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Debra',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([1940, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952,
                          1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,
                          1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976,
                          1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988,
                          1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
                          2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2012, 2013, 2016],
                         dtype=int64),
              'xaxis': 'x',
              'y': array([   7,    7,    8,   15,   19,   20,   56,   92,  199,  601, 1510, 2351,
                          3295, 3784, 3969, 3755, 3318, 2660, 2290, 2014, 1647, 1592, 1430, 1287,
                          1154,  958,  818,  748,  647,  547,  463,  318,  242,  236,  159,  151,
                           151,  164,  130,  141,   97,  114,   97,   95,   93,   64,   78,   69,
                            71,   51,   62,   41,   34,   28,   28,   12,   14,   16,   10,   13,
                            14,   10,    7,   12,   13,   12,   13,    6,    7,    5,    8,    5],
                         dtype=int64),
              'yaxis': 'y'}],
    'layout': {'font': {'size': 18},
               'height': 400,
               'legend': {'title': {'text': 'Name'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Popularity for: Debra'},
               'width': 1000,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Year'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Count'}}}
})

We can visualize the decrease in the popularity of the name "Debra:"

In [8]:
import plotly.express as px
def plot_name(*names):

    fig = px.line(f_babynames[f_babynames["Name"].isin(names)],
                  x = "Year", y = "Count", color="Name",
                  title=f"Popularity for: {names}")
    fig.update_layout(font_size = 18,
                  width=1000,
                  height=400)
    return fig

plot_name("Debra")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Name=Debra<br>Year=%{x}<br>Count=%{y}<extra></extra>',
              'legendgroup': 'Debra',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Debra',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([1940, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952,
                          1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964,
                          1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976,
                          1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988,
                          1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
                          2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2012, 2013, 2016],
                         dtype=int64),
              'xaxis': 'x',
              'y': array([   7,    7,    8,   15,   19,   20,   56,   92,  199,  601, 1510, 2351,
                          3295, 3784, 3969, 3755, 3318, 2660, 2290, 2014, 1647, 1592, 1430, 1287,
                          1154,  958,  818,  748,  647,  547,  463,  318,  242,  236,  159,  151,
                           151,  164,  130,  141,   97,  114,   97,   95,   93,   64,   78,   69,
                            71,   51,   62,   41,   34,   28,   28,   12,   14,   16,   10,   13,
                            14,   10,    7,   12,   13,   12,   13,    6,    7,    5,    8,    5],
                         dtype=int64),
              'yaxis': 'y'}],
    'layout': {'font': {'size': 18},
               'height': 400,
               'legend': {'title': {'text': 'Name'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': "Popularity for: ('Debra',)"},
               'width': 1000,
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Year'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Count'}}}
})

In [59]:
# Find the 10 names that have decreased the most in popularity
top10=aq.value_counts("Count RTP").head(10).index
top10

Index([5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64', name='Count RTP')

In [60]:
plot_name(*top10)

TypeError: sequence item 0: expected str instance, int found

For fun, try plotting your name or your friends' names.

### Slido Exercise

Given the example below on `babynames` dataset, write code to compute the total number of babies with each name in California using with and without agg.

In [64]:
# code here
# Using the same filter for California
california_names = babynames[babynames['State'] == 'CA']

# Group by 'Name' and aggregate with sum on 'Count'
total_counts_agg = california_names.groupby('Name').agg({'Count': 'sum'})
print(total_counts_agg)

         Count
Name          
Aadan       18
Aadarsh      6
Aaden      647
Aadhav      27
Aadhini      6
...        ...
Zymir        5
Zyon       133
Zyra       103
Zyrah       21
Zyrus        5

[20437 rows x 1 columns]


### Slido Exercise

Write code to compute the total number of babies born each year in California.

In [9]:
# code here
# Assuming 'babynames' DataFrame has columns 'Year', 'State', and 'Count'
# and you want to filter by the state of California (CA)

# Filter the DataFrame for California
california_babies = babynames[babynames['State'] == 'CA']

# Group by 'Year' and sum the 'Count'
yearly_totals = california_babies.groupby('Year')['Count'].sum()
print(yearly_totals)

Year
1910      9163
1911      9983
1912     17946
1913     22094
1914     26926
         ...  
2018    395436
2019    386996
2020    362882
2021    362582
2022    360023
Name: Count, Length: 113, dtype: int64


In [10]:
# Plotting baby counts per year
babynames['Year']

0         1910
1         1910
2         1910
3         1910
4         1910
          ... 
407423    2022
407424    2022
407425    2022
407426    2022
407427    2022
Name: Year, Length: 407428, dtype: int64

### `groupby.size` and `groupby.count()`

In [4]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'letter':['A', 'A', 'B', 'C', 'C', 'C'],
                   'num':[1, 2, 3, 4, np.NaN, 4],
                   'state':[np.NaN, 'tx', 'fl', 'hi', np.NaN, 'ak']})
df

,letter,num,state
0,A,1.0,NaN
1,A,2.0,tx
2,B,3.0,fl
3,C,4.0,hi
4,C,NaN,NaN
5,C,4.0,ak


`groupby.size()` returns a `Series`, indexed by the `letter`s that we grouped by, with values denoting the number of rows in each group/sub-DataFrame. It does not care about missing (`NaN`) values.

In [5]:
# Use groupby with size()
# Group by 'letter' and calculate the size of each group
group_sizes = df.groupby('letter').size()

# Display the result
print(group_sizes)


letter
A    2
B    1
C    3
dtype: int64


`groupby.count()` returns a `DataFrame`, indexed by the `letter`s that we grouped by. Each column represents the number of non-missing values for that `letter`.

In [6]:
# Use groupby with count()
# Group by 'letter' and count non-missing values in each column
group_counts = df.groupby('letter').count()

# Display the result
print(group_counts)


        num  state
letter            
A         2      1
B         1      1
C         2      2


You might recall `value_counts()` function we talked about last week. What's the difference?

In [7]:
# Use value_count() on DataFrame described above
# Apply value_counts() to each column of the DataFrame
letter_counts = df['letter'].value_counts(dropna=False)
num_counts = df['num'].value_counts(dropna=False)
state_counts = df['state'].value_counts(dropna=False)

# Display the results
print("Letter Counts:")
print(letter_counts)
print("\nNum Counts:")
print(num_counts)
print("\nState Counts:")
print(state_counts)


Letter Counts:
letter
C    3
A    2
B    1
Name: count, dtype: int64

Num Counts:
num
4.0    2
1.0    1
2.0    1
3.0    1
NaN    1
Name: count, dtype: int64

State Counts:
state
NaN    2
tx     1
fl     1
hi     1
ak     1
Name: count, dtype: int64


Turns out `value_counts()` does something similar to `groupby.size()`, except that it also sorts the resulting `Series` in descending order.

## Filtering by Group

In [1]:
# Let's read the elections dataset
import pandas as pd
election = pd.read_csv(r'C:\Users\pc\Desktop\Data Science with ML\pandas\elections.csv')


Let's keep only the elections years where the maximum vote share `%` is less than 45%.

In [3]:
# use filter function
election.groupby("Year").filter(lambda sf: sf["%"].max() < 45)

,Candidate,Party,%,Year,Result
7,Clinton,Democratic,43.0,1992,win
8,Bush,Republican,37.4,1992,loss
9,Perot,Independent,18.9,1992,loss


### `groupby` Puzzle

Assume that we want to know the best election by each party.

#### Attempt #1

We have to be careful when using aggregation functions. For example, the code below might be misinterpreted to say that Woodrow Wilson successfully ran for election in 2020. Why is this happening?

In [5]:
# Use agg(max)
result = election.groupby('Year').agg({'%': 'max'})

print(result)

         %
Year      
1980  50.7
1984  58.8
1988  53.4
1992  43.0
1996  49.2
2000  48.4
2004  50.7
2008  52.9
2012  51.1
2016  48.2


#### Attempt #2

Next, we'll write code that properly returns _the best result by each party_. That is, each row should show the Year, Candidate, Popular Vote, Result, and % for the election in which that party saw its best results (rather than mixing them as in the example above). Here's what the first rows of the correct output should look like:

![parties.png](attachment:ab21f8de-ad29-46c2-bea7-e9aea9c40e3e.png)

In [9]:
import pandas as pd

# Assuming 'election' is your DataFrame and it has columns 'Year', 'Candidate', 'Popular Vote', 'Result', 'Party', and '%'
# Read the CSV file (if not already read)
election = pd.read_csv(r'C:\Users\pc\Desktop\Data Science with ML\pandas\elections.csv')

# Sort the DataFrame by 'Party' and '%', descending order so the highest % is first
sorted_election = election.sort_values(by=['Party', '%'], ascending=[True, False])

# Group by 'Party' and take the first row of each group
best_results_by_party = sorted_election.groupby('Party').first().reset_index()

# The resulting DataFrame 'best_results_by_party' will have the best result for each party
print(best_results_by_party)

         Party Candidate     %  Year Result
0   Democratic     Obama  52.9  2008    win
1  Independent     Perot  18.9  1992   loss
2   Republican    Reagan  58.8  1984    win


In [11]:
sorted_election.groupby("Party").first()

,Candidate,%,Year,Result
Party,,,,
Democratic,Obama,52.9,2008,win
Independent,Perot,18.9,1992,loss
Republican,Reagan,58.8,1984,win


#### Alternative Solutions

You'll soon discover that with Pandas rich tool set, there's typically more than one way to get to the same answer. Each approach has different tradeoffs in terms of readability, performance, memory consumption, complexity, and more. It will take some experience for you to develop a sense of which approach is better for each problem, but you should, in general, try to think if you can at least envision a different solution to a given problem, especially if you find your current solution to be particularly convoluted or hard to read.

Here are a couple of other ways of obtaining the same result (in each case, we only show the top part with `head()`). The first approach uses `groupby` but finds the location of the maximum value via the `idxmax()` method (look up its documentation!).  We then index and sort by `Party` to match the requested formatting:

In [12]:
# Use idxmax function
idx = election.groupby('Party')['%'].idxmax()

# Use the index to get the rows with the best result for each party
best_results_by_party = election.loc[idx].sort_values(by='Party')

# Display the top part of the DataFrame
best_results_by_party.head()

,Candidate,Party,%,Year,Result
17,Obama,Democratic,52.9,2008,win
9,Perot,Independent,18.9,1992,loss
3,Reagan,Republican,58.8,1984,win


In [13]:
# This is the computational part, Extract DataFrame based on above mentioned condition


# Apply sort_index to match the formatting above
idx = election.groupby('Party')['%'].idxmax()

# Use the index to get the rows with the best result for each party
best_results_by_party = election.loc[idx]

# Apply sort_index to match the formatting above
best_results_by_party_sorted = best_results_by_party.sort_index()

# Display the top part of the DataFrame
best_results_by_party_sorted.head()

,Candidate,Party,%,Year,Result
3,Reagan,Republican,58.8,1984,win
9,Perot,Independent,18.9,1992,loss
17,Obama,Democratic,52.9,2008,win


Another approach is listed below. And this one doesn't even use `groupby`!

This approach instead uses the `drop_duplicates` method to keep only the last occurrence of of each party after having sorted by "%", which is the best performance.

In [16]:

# code here
import pandas as pd

# Assuming 'election' is your DataFrame and it has columns 'Year', 'Candidate', 'Popular Vote', 'Result', 'Party', and '%'
# Read the CSV file (if not already read)
election = pd.read_csv(r'C:\Users\pc\Desktop\Data Science with ML\pandas\elections.csv')

# Sort the DataFrame by 'Party' and '%', descending order so the highest % is first
sorted_election = election.sort_values(by=['Party', '%'], ascending=[True, False])

# Use drop_duplicates to keep only the last occurrence of each party
best_performance_by_party = sorted_election.drop_duplicates(subset='Party', keep='last')

# The resulting DataFrame 'best_performance_by_party' will have the best result for each party
best_performance_by_party

,Candidate,Party,%,Year,Result
4,Mondale,Democratic,37.6,1984,loss
2,Anderson,Independent,6.6,1980,loss
8,Bush,Republican,37.4,1992,loss


*Challenge:* See if you can find yet another approach that still gives the same answer.

### `DataFrameGroupBy` Objects

The result of `groupby` is not a `DataFrame` or a list of `DataFrame`s. It is instead a special type called a `DataFrameGroupBy`.

In [19]:
grouped_by_party = election.groupby("Party")
type(grouped_by_party)
election

,Candidate,Party,%,Year,Result
0,Reagan,Republican,50.7,1980,win
1,Carter,Democratic,41.0,1980,loss
2,Anderson,Independent,6.6,1980,loss
3,Reagan,Republican,58.8,1984,win
4,Mondale,Democratic,37.6,1984,loss
5,Bush,Republican,53.4,1988,win
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
8,Bush,Republican,37.4,1992,loss
9,Perot,Independent,18.9,1992,loss


`GroupBy` objects are structured like dictionaries. In fact, we can actually see the dictionaries with the following code:

In [20]:
# visualize groups
# Assuming 'election' is your DataFrame and it has a 'Party' column
grouped_by_party = election.groupby("Party")

# Iterate over each group
for party_name, group in grouped_by_party:
    print(f"Party: {party_name}")
    print(group.head())  # Print the first few rows of each group
    print("\n")  # Add a newline for readability between groups


Party: Democratic
   Candidate       Party     %  Year Result
1     Carter  Democratic  41.0  1980   loss
4    Mondale  Democratic  37.6  1984   loss
6    Dukakis  Democratic  45.6  1988   loss
7    Clinton  Democratic  43.0  1992    win
10   Clinton  Democratic  49.2  1996    win


Party: Independent
   Candidate        Party     %  Year Result
2   Anderson  Independent   6.6  1980   loss
9      Perot  Independent  18.9  1992   loss
12     Perot  Independent   8.4  1996   loss


Party: Republican
   Candidate       Party     %  Year Result
0     Reagan  Republican  50.7  1980    win
3     Reagan  Republican  58.8  1984    win
5       Bush  Republican  53.4  1988    win
8       Bush  Republican  37.4  1992   loss
11      Dole  Republican  40.7  1996   loss




The `key`s of the dictionary are the groups (in this case, `Party`), and the `value`s are the **indices** of rows belonging to that group. We can access a particular sub-`DataFrame` using `get_group`:

In [22]:
# code here
# Assuming 'grouped_by_party' is a DataFrameGroupBy object created from the 'election' DataFrame
grouped_by_party = election.groupby("Party")

# Access the sub-DataFrame for a specific party, say 'Democratic'
democratic_group = grouped_by_party.get_group('Democratic')

# Now 'democratic_group' contains only the rows from 'election' where the 'Party' is 'Democratic'
democratic_group

,Candidate,Party,%,Year,Result
1,Carter,Democratic,41.0,1980,loss
4,Mondale,Democratic,37.6,1984,loss
6,Dukakis,Democratic,45.6,1988,loss
7,Clinton,Democratic,43.0,1992,win
10,Clinton,Democratic,49.2,1996,win
13,Gore,Democratic,48.4,2000,loss
15,Kerry,Democratic,48.3,2004,loss
17,Obama,Democratic,52.9,2008,win
19,Obama,Democratic,51.1,2012,win
21,Clinton,Democratic,48.2,2016,loss


---